In [1]:
import asyncio
import logging
import struct
import sys
import platform
from datetime import datetime
from bleak import BleakClient, BleakScanner, BleakError
from bleak.backends.device import BLEDevice
from bleak.backends.scanner import AdvertisementData

# Configuración de logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("ble_interactor.log"),
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)

# Clase para manejar el registro de datos
class DataLogger:
    def __init__(self, filename="data_log.txt"):
        self.filename = filename

    def log(self, message):
        with open(self.filename, "a") as file:
            file.write(f"{datetime.now()} - {message}\n")

# Clase principal para interactuar con dispositivos BLE
class BLEInteractor:
    def __init__(self):
        self.devices = []
        self.client = None
        self.max_retries = 3
        self.retry_delay = 2
        self.data_logger = DataLogger()
        self.connected_device = None

        # UUIDs comunes
        self.BATTERY_SERVICE_UUID = "0000180F-0000-1000-8000-00805F9B34FB"
        self.BATTERY_LEVEL_UUID = "00002A19-0000-1000-8000-00805F9B34FB"
        self.MANUFACTURER_NAME_UUID = "00002A29-0000-1000-8000-00805F9B34FB"
        self.MODEL_NUMBER_UUID = "00002A24-0000-1000-8000-00805F9B34FB"
        self.CURRENT_TIME_UUID = "00002A2B-0000-1000-8000-00805F9B34FB"

        # Diccionario para mantener manejadores de notificaciones
        self.notification_handlers = {}

    async def scan_devices(self, timeout=5.0):
        """Escanea dispositivos BLE cercanos y guarda la lista ordenada por RSSI."""
        try:
            logger.info("Buscando dispositivos BLE cercanos...")

            # Ahora especificamos return_adv=True para obtener AdvertisementData
            found = await BleakScanner.discover(timeout=timeout, return_adv=True)
            print(found)

            # 'found' es una lista de tuplas (device, advertisement_data)
            # Ordenar dispositivos por RSSI (intensidad de señal)
            self.devices = sorted(
                found.values(),
                key=lambda x: x[1].rssi if x[1].rssi else -100,
                reverse=True
            )
            return self.devices
        except Exception as e:
            logger.error(f"Error durante el escaneo: {e}")
            return []

    def display_devices(self):
        """Muestra los dispositivos encontrados con metadatos."""
        if not self.devices:
            logger.info("No se encontraron dispositivos.")
            return

        print("\nDispositivos encontrados:")
        for idx, (device, advertisement_data) in enumerate(self.devices):
            display_str = f"""
DEVICE {idx}:
    Name               - {device.name or 'Desconocido'}
    Address            - {device.address}
    RSSI               - {advertisement_data.rssi}
    Manufacturer Data  - {advertisement_data.manufacturer_data}
    Service UUIDs      - {advertisement_data.service_uuids}
    """
            print(display_str)

    async def connect_to_device(self, device):
        """Conecta al dispositivo BLE seleccionado con manejo de emparejamiento."""
        for attempt in range(self.max_retries):
            try:
                logger.info(f"Intentando conectar a {device.address} (Intento {attempt + 1})")
                # En Linux, el emparejamiento no es soportado a través de bleak
                client = BleakClient(device, timeout=30)
                await client.connect(timeout=10.0)
                if client.is_connected:
                    logger.info(f"Conectado a {device.address}")
                    self.connected_device = device
                    return client
            except Exception as e:
                logger.error(f"Intento {attempt + 1} fallido: {str(e)}")
                await asyncio.sleep(self.retry_delay)
        raise ConnectionError(f"No se pudo conectar al dispositivo después de {self.max_retries} intentos")

    async def connect(self, device):
        """Conecta al dispositivo y guarda el cliente para futuras interacciones."""
        self.client = await self.connect_to_device(device)

    async def disconnect(self):
        """Desconecta del dispositivo."""
        if self.client and self.client.is_connected:
            await self.client.disconnect()
            logger.info("Desconectado del dispositivo.")
            self.client = None
            self.connected_device = None

    async def read_battery_level(self):
        """Lee el nivel de batería del dispositivo."""
        if not self.client or not self.client.is_connected:
            logger.error("No hay conexión con ningún dispositivo.")
            return None
        try:
            value = await self.client.read_gatt_char(self.BATTERY_LEVEL_UUID)
            battery_level = int(value[0])
            logger.info(f"Nivel de batería: {battery_level}%")
            self.data_logger.log(f"Nivel de batería: {battery_level}%")
            return battery_level
        except Exception as e:
            logger.error(f"Error al leer nivel de batería: {e}")
            return None

    async def read_device_info(self):
        """Lee información básica del dispositivo."""
        if not self.client or not self.client.is_connected:
            logger.error("No hay conexión con ningún dispositivo.")
            return None
        try:
            manufacturer = await self.client.read_gatt_char(self.MANUFACTURER_NAME_UUID)
            model = await self.client.read_gatt_char(self.MODEL_NUMBER_UUID)
            info = {
                "manufacturer": manufacturer.decode(errors='replace'),
                "model": model.decode(errors='replace')
            }
            logger.info(f"Información del dispositivo: {info}")
            self.data_logger.log(f"Información del dispositivo: {info}")
            return info
        except Exception as e:
            logger.error(f"Error al leer información del dispositivo: {e}")
            return None

    async def read_current_time(self):
        """Lee el tiempo actual del dispositivo."""
        if not self.client or not self.client.is_connected:
            logger.error("No hay conexión con ningún dispositivo.")
            return None
        try:
            value = await self.client.read_gatt_char(self.CURRENT_TIME_UUID)
            year, month, day, hours, minutes, seconds = struct.unpack('<HBBBBB', value[:7])
            current_time = datetime(year, month, day, hours, minutes, seconds)
            logger.info(f"Tiempo del dispositivo: {current_time}")
            self.data_logger.log(f"Tiempo del dispositivo: {current_time}")
            return current_time
        except Exception as e:
            logger.error(f"Error al leer tiempo actual: {e}")
            return None

    async def get_all_characteristics(self):
        """Obtiene todas las características disponibles del dispositivo."""
        if not self.client or not self.client.is_connected:
            logger.error("No hay conexión con ningún dispositivo.")
            return None
        try:
            services = await self.client.get_services()
            all_characteristics = {}

            for service in services:
                service_chars = []
                for char in service.characteristics:
                    char_info = {
                        "uuid": str(char.uuid),
                        "description": char.description,
                        "properties": list(char.properties)
                    }
                    service_chars.append(char_info)

                all_characteristics[str(service.uuid)] = {
                    "description": service.description,
                    "characteristics": service_chars
                }

            logger.info("Características obtenidas del dispositivo.")
            self.data_logger.log(f"Características: {all_characteristics}")
            return all_characteristics
        except Exception as e:
            logger.error(f"Error al obtener características: {e}")
            return None

    def notification_handler(self, sender, data):
        """Manejador genérico para notificaciones."""
        logger.info(f"Notificación de {sender}: {data}")
        self.data_logger.log(f"Notificación de {sender}: {data}")

        # Procesamiento específico según UUID
        uuid = str(sender.uuid)
        if uuid in self.notification_handlers:
            self.notification_handlers[uuid](sender, data)
        else:
            logger.info(f"No hay manejador específico para UUID: {uuid}")

    async def subscribe_to_notifications(self, uuid):
        """Suscribe a notificaciones de una característica específica."""
        if not self.client or not self.client.is_connected:
            logger.error("No hay conexión con ningún dispositivo.")
            return False
        try:
            await self.client.start_notify(uuid, self.notification_handler)
            logger.info(f"Suscrito a notificaciones de {uuid}")
            return True
        except Exception as e:
            logger.error(f"Error al suscribirse a notificaciones en {uuid}: {e}")
            return False

    def register_notification_handler(self, uuid, handler):
        """Registra un manejador específico para una característica."""
        self.notification_handlers[uuid] = handler
        logger.info(f"Manejador registrado para UUID: {uuid}")

    async def send_media_command(self, command):
        """Envía un comando multimedia al dispositivo."""
        if not self.client or not self.client.is_connected:
            logger.error("No hay conexión con ningún dispositivo.")
            return False

        MEDIA_REMOTE_COMMAND_UUID = "9b3c81d8-57b1-4a8a-b8df-0e56f7ca51c2"
        commands = {
            'play': b'\x01',
            'pause': b'\x02',
            'next': b'\x03',
            'previous': b'\x04',
            'volume_up': b'\x05',
            'volume_down': b'\x06'
        }

        if command not in commands:
            logger.error(f"Comando no válido: {command}")
            return False

        try:
            await self.client.write_gatt_char(
                MEDIA_REMOTE_COMMAND_UUID,
                commands[command],
                response=True
            )
            logger.info(f"Comando '{command}' enviado exitosamente.")
            self.data_logger.log(f"Comando '{command}' enviado.")
            return True
        except Exception as e:
            logger.error(f"Error al enviar comando multimedia: {e}")
            return False

    # Función para manejar dispositivos que requieren emparejamiento o autenticación
    async def pair_device(self):
        """Intenta emparejar el dispositivo."""
        if not self.client or not self.connected_device:
            logger.error("No hay dispositivo para emparejar.")
            return False

        if platform.system() == "Windows" or platform.system() == "Darwin":
            try:
                paired = await self.client.pair(protection_level=2)  # Nivel de protección 2 para autenticación
                if paired:
                    logger.info("Dispositivo emparejado exitosamente.")
                    return True
                else:
                    logger.error("El emparejamiento falló.")
                    return False
            except Exception as e:
                logger.error(f"Error durante el emparejamiento: {e}")
                return False
        else:
            logger.warning("El emparejamiento no está soportado en esta plataforma.")
            return False

    # Ejemplo de manejador específico para notificaciones de temperatura
    def temperature_notification_handler(self, sender, data):
        """Procesa notificaciones de temperatura."""
        # Suponiendo que la temperatura viene en formato float de 4 bytes
        temperature = struct.unpack('<f', data)[0]
        logger.info(f"Temperatura recibida: {temperature}°C")
        self.data_logger.log(f"Temperatura recibida: {temperature}°C")

    # Ejemplo de manejador específico para notificaciones de acelerómetro
    def accelerometer_notification_handler(self, sender, data):
        """Procesa notificaciones de acelerómetro."""
        # Suponiendo que los datos vienen en tres floats consecutivos
        x, y, z = struct.unpack('<fff', data)
        logger.info(f"Acelerómetro - X: {x}, Y: {y}, Z: {z}")
        self.data_logger.log(f"Acelerómetro - X: {x}, Y: {y}, Z: {z}")

# Función para manejar entrada asíncrona
async def async_input(prompt):
    print(prompt, end='', flush=True)
    return await asyncio.get_event_loop().run_in_executor(None, sys.stdin.readline)

# Sesión interactiva
async def interactive_session():
    ble = BLEInteractor()

    # Escanear dispositivos
    devices = await ble.scan_devices()
    print("DVs0",devices)
    if not devices:
        return

    ble.display_devices()

    try:
        idx = int(input("\nSelecciona el índice del dispositivo: "))
        device = ble.devices[idx]
        print("D0",device)
        await ble.connect(device[0])
    except Exception as e:
        logger.error(f"Error al conectar: {e}")
        return

    # Intentar emparejar el dispositivo
    await ble.pair_device()

    # Menú interactivo
    while True:
        print("\nOpciones disponibles:")
        print("1. Leer nivel de batería")
        print("2. Suscribirse a notificaciones de batería")
        print("3. Leer información del dispositivo")
        print("4. Leer tiempo actual")
        print("5. Enviar comando multimedia")
        print("6. Ver todas las características")
        print("7. Suscribirse a notificaciones avanzadas")
        print("8. Desconectar y salir")

        choice = input("\nSelecciona una opción: ")

        if choice == "1":
            level = await ble.read_battery_level()
            if level is not None:
                print(f"Nivel de batería: {level}%")

        elif choice == "2":
            await ble.subscribe_to_notifications(ble.BATTERY_LEVEL_UUID)
            print("Suscrito a notificaciones de batería.")

        elif choice == "3":
            info = await ble.read_device_info()
            if info:
                print(f"Fabricante: {info['manufacturer']}")
                print(f"Modelo: {info['model']}")

        elif choice == "4":
            time = await ble.read_current_time()
            if time:
                print(f"Tiempo del dispositivo: {time}")

        elif choice == "5":
            print("\nComandos disponibles:")
            print("play, pause, next, previous, volume_up, volume_down")
            cmd = input("Introduce el comando: ").strip()
            success = await ble.send_media_command(cmd)
            if success:
                print("Comando enviado exitosamente.")

        elif choice == "6":
            characteristics = await ble.get_all_characteristics()
            if characteristics:
                print("\nCaracterísticas disponibles:")
                for service_uuid, service_info in characteristics.items():
                    print(f"\nServicio: {service_uuid}")
                    print(f"Descripción: {service_info['description']}")
                    print("Características:")
                    for char in service_info['characteristics']:
                        print(f"  UUID: {char['uuid']}")
                        print(f"  Descripción: {char['description']}")
                        print(f"  Propiedades: {', '.join(char['properties'])}")

        elif choice == "7":
            # Suscribirse a notificaciones avanzadas
            characteristics = await ble.get_all_characteristics()
            if not characteristics:
                continue

            # Mostrar características con propiedad 'notify'
            notify_chars = []
            for service_uuid, service_info in characteristics.items():
                for char in service_info['characteristics']:
                    if 'notify' in char['properties']:
                        notify_chars.append(char)

            if not notify_chars:
                print("No se encontraron características que admitan notificaciones.")
                continue

            print("\nCaracterísticas que admiten notificaciones:")
            for idx, char in enumerate(notify_chars):
                print(f"{idx}. UUID: {char['uuid']}, Descripción: {char['description']}")

            char_idx = int(input("Selecciona el índice de la característica: "))
            selected_char = notify_chars[char_idx]

            # Registrar manejador específico si es necesario
            if 'temperature' in selected_char['description'].lower():
                ble.register_notification_handler(selected_char['uuid'], ble.temperature_notification_handler)
            elif 'accelerometer' in selected_char['description'].lower():
                ble.register_notification_handler(selected_char['uuid'], ble.accelerometer_notification_handler)

            # Suscribirse a las notificaciones
            success = await ble.subscribe_to_notifications(selected_char['uuid'])
            if success:
                print(f"Suscrito a notificaciones de {selected_char['uuid']}.")

        elif choice == "8":
            await ble.disconnect()
            break

        else:
            print("Opción no válida. Por favor, intenta de nuevo.")

In [2]:
try:
    await interactive_session()
except KeyboardInterrupt:
        print("\nPrograma terminado por el usuario.")

2024-11-09 01:05:26,827 - INFO - Buscando dispositivos BLE cercanos...
{'20:AE:84:C4:BC:05': (BLEDevice(20:AE:84:C4:BC:05, 20-AE-84-C4-BC-05), AdvertisementData(manufacturer_data={6: b'\x01\t!\n]\xc4}\xe7\x1cpDESKTOP-GVE588D'}, rssi=-87))}
DVs0 [(BLEDevice(20:AE:84:C4:BC:05, 20-AE-84-C4-BC-05), AdvertisementData(manufacturer_data={6: b'\x01\t!\n]\xc4}\xe7\x1cpDESKTOP-GVE588D'}, rssi=-87))]

Dispositivos encontrados:

DEVICE 0:
    Name               - 20-AE-84-C4-BC-05
    Address            - 20:AE:84:C4:BC:05
    RSSI               - -87
    Manufacturer Data  - {6: b'\x01\t!\n]\xc4}\xe7\x1cpDESKTOP-GVE588D'}
    Service UUIDs      - []
    
D0 (BLEDevice(20:AE:84:C4:BC:05, 20-AE-84-C4-BC-05), AdvertisementData(manufacturer_data={6: b'\x01\t!\n]\xc4}\xe7\x1cpDESKTOP-GVE588D'}, rssi=-87))
2024-11-09 01:05:34,759 - INFO - Intentando conectar a 20:AE:84:C4:BC:05 (Intento 1)
2024-11-09 01:05:34,763 - ERROR - Intento 1 fallido: device 'dev_20_AE_84_C4_BC_05' not found
2024-11-09 01:05:36,